# `RATstart` 

This section deals with using the direct interface to the `librat` code at a high level, through the `RATstart` software. This is a short piece of `C` code, the main role of with is to present a set of simple functions controlled by text input on the command line. One of these 'options' (option 14) gives an extremely fexible interface for defining viewing and illumination conditions for radiative transfer simulations, and it is this part of the interface that is most widely used. 

In this section, we introduce some of the functionality of `RATstart`. We use a `bash` environment here around `RATstart` to emphasise skills and code in running this from a standard `*nix` command lines. We use a python3 environment for plotting and related codes. 

We first set some local variables.

In [1]:
import os
import sys
path = os.environ['PATH'].split(':')
path.extend(sys.path)
sys.path = path

In [12]:
os.environ['DYLD_LIBRARY_PATH']

'/Users/plewis/Documents/GitHub/librat/src:'

In [9]:
from RATsetup import *

## `RATstart` (formerly `start`)


`RATstart` used to be called `start`, but that causes conflicts sometimes, so its now changed to `RATstart`.

### Getting started

The minimum requirement for the `RATstart` command line is the declaration of some object file to use in the simulation. An object file describes the real world geometry and scattering properties of the objects we are using in the simulation, such as some representation of a vegetation canopy. We generate a very simple object file for demonstrating `RATstart`:

    !{
    usemtl WHITE
    v 0 0 0 
    sph -1 100
    !}

Other than that, its functionality is mainly controlled though a series of option codes read on the input. 

Valid options are (currently) between 0 and 16, inclusive, with some missing. You shouldn't use the ones not listed as they tend to be experimental and may not work as expected.

If we give a non-valid option code, we get the help message:

In [11]:
%%bash 

# a simple object file
# and invalid code: 1000 
# to see the options
# a very simple object file
cat <<EOF > tmp.$$.obj
!{
usemtl WHITE
v 0 0 0 
sph -1 100
!}
EOF

#export LD_LIBRARY_PATH="${BPMS}/src:${LD_LIBRARY_PATH}"
#export DYLD_LIBRARY_PATH="${BPMS}/src:${DYLD_LIBRARY_PATH}"
RATstart tmp.$$.obj <<< 1000; 

# tidy up, removing file 
rm -f $BPMSROOT/tmp.$$.obj

dyld: Library not loaded: libratlib.so
  Referenced from: /Users/plewis/Documents/GitHub/librat/bin/x86_64/RATstart
  Reason: image not found
bash: line 16: 21547 Abort trap: 6           RATstart tmp.$$.obj <<< 1000


As noted above, we most commonly use option `14`. This allows the definition of a 'camera' and 'light source' defined in the files `camera.dat` and `light.dat` here, respectively and gives a very general interface for radiative transfer simulations. Using the code then amounts to setting up camera and illumination files for the scenarious you are interested in, which greatly simplifies the interface. 

The command line option `-RAThelp`  will provide the list of command line options:

In [ ]:
%%bash 
# a simple object file
# --help to see command line options
# a very simple object file
cat <<EOF > $BPMSROOT/tmp.$$.obj
!{
usemtl WHITE
v 0 0 0 
sph -1 100
!}
EOF

export LD_LIBRARY_PATH="$BPMS/src:${LD_LIBRARY_PATH}"
export DYLD_LIBRARY_PATH="$BPMS/src:${DYLD_LIBRARY_PATH}"
RATstart -RAThelp tmp.$$.obj <<< 1000; 

# tidy up, removing file 
rm -f $BPMSROOT/tmp.$$.obj

In the rest of this section, we will learn how to make a call to `RATstart` and how to use the various command line options and input options.



### Initialisation shells

Before proceeding, we make sure we set appropriate environment variables to run a set of examples with RATstart. We also want to test that the software runs ok. We generate a `bash` shell to achieve this here, and use this in subsequent notes. 

You can modify any of this as a user, but be careful not to break it! 🙄. In an emergency, you can always just re-download this set of notes and software from [github](https://github.com/profLewis/librat) and start again.

If you want to know more about setting up this sort of file, and what it all means, see [Appendix 1](Appendix1). At the moment, you just need to be aware of the need to set environment variables, but not to understand all of the intricacies.

We simplify the interface for the environment variables by setting up the configuration file `examples_init.sh`. We then run it, to test the code.

In [ ]:
%%bash
#
# create examples_init.sh
# for examples initialisation
#
# create the init shell
cat <<EOF > $BPMS/bin/examples_init.sh
#!/bin/bash
#
# defaults
#
export BPMS=\${BPMS-$BPMS}
export BPMSROOT=\${BPMSROOT-\$BPMSROOT}
lib=\${lib-\$BPMS/src}
bin=\${bin-\$BPMS/src}
VERBOSE=\${VERBOSE-0}
export TEMP=\${TEMP-/tmp}

# set up required environment variables for bash
export LD_LIBRARY_PATH="\${lib}:\${LD_LIBRARY_PATH}"
export DYLD_LIBRARY_PATH="\${lib}:\${DYLD_LIBRARY_PATH}"
export PATH="\${bin}:\${PATH}"

export MATLIB=\$BPMSROOT
export RSRLIB=\$BPMSROOT
export ARARAT_OBJECT=\$BPMSROOT
export DIRECT_ILLUMINATION=\$BPMSROOT
export BPMS_FILES=\$BPMSROOT
export SKY_ILLUMINATION==\$BPMSROOT

if [ "\$(which RATstart)" == "\${bin}/RATstart" ]
then
  if [ "\$VERBOSE" == 1 ]; then
      echo "RATstart found ok"
  fi
else
  # we should create them
  make clean all 
fi
EOF
chmod +x $BPMS/bin/examples_init.sh

Let us set the `VERBOSE` flag on, by setting:

    export VERBOSE=1
    
before running `$INIT`. This will cause the shell to print `RATstart found ok` is the executable `RATstart` is where it is expected.

In [ ]:
%%bash
# run test
export VERBOSE=1
source examples_init.sh

The shell sets `BPMSROOT` to be `$BPMS/obj` by default. This causes other variables such as `MATLIB` to be set to the same value, so `RATstart` will look for materials in `$BPMS/obj` (see below):

In [ ]:
%%bash

# run test
source examples_init.sh
echo "MATLIB is $MATLIB"

If we want to use this shell for `BPMSROOT` elsewhere, such as `/home/me/mydata/somewhere`, we just need to make sure `BPMSROOT` is set to this before running `$INIT`:

In [ ]:
%%bash

# run test
export BPMSROOT=/home/me/mydata/somewhere
source examples_init.sh
echo "MATLIB is $MATLIB"

## Environment variables

If we take a quick look at the `examples_init.sh` file we generated, we see:

In [ ]:
cat $BPMS/bin/examples_init.sh


We might notice that `examples_init.sh` sets a number of environment variables, namely `MATLIB`, `RSRLIB` etc. 

If you are interested in the details, the meaning of these is given in the table below. 

Alternatively, just notice that in the init shell, all of these variables are set to  `BPMSROOT`, so if we want to point to the location of an object and material database for librat, we need only set the environment variable  `BPMSROOT` appropriately. In this case it defaults to `$BPMS/obj`.






**Table explaining librat object environment variables:**


| Name | File types |
|:-:|:-:|
| `MATLIB` | material library e.g. `plants.matlib`, all materials defined in a material library e.g. `white.dat`|
| `ARARAT_OBJECT` | (extended) wavefront object files e.g. `first.obj`  |
| `DIRECT_ILLUMINATION` | spectral files for direct illumination: those defined in `-RATdirect` command line option |
| `RSRLIB` | sensor waveband files: those defined in `-RATsensor_wavebands` command line option |
| `BPMS_FILES` | Not used |
| `SKY_ILLUMINATION` | location of sky map image files: defined in `-RATskymap`  command line option |


You can set all of these to the same value  as above (`$BPMSROOT`), in which case the database of files is all defined relative to that location in the file system. This is the most typical use of `librat`. We illustrate this setup below for the `librat` distribution, where a set of examples use files from the directory `test/test_example`.

Additionally in `librat`, the following environment variables can be set to extend the size of some aspects of the model. You would only need to use these in some extreme case.

**Table explaining additional librat environment variables:**


| Name | Purpose |
|:-:|:-:|
| `MAX_GROUPS` | Maximum number of groups allowed (100000) |
| `PRAT_MAX_MATERIALS` | Maximum number of materials allowed (DEFAULT_PRAT_MAX_MATERIALS=1024 in `mtllib.h`) |
| `MAX_SUNS` | Maximum number of suns (180 in `rat.h`) |



If you want to run a shell that uses the setup, you will need to use the command `source` to export the variables to your shell.

You can test the init file by running the cell (shell) below. 

In [ ]:
%%bash

# source if we want the info in this shell
source examples_init.sh

echo "MATLIB is set to $MATLIB"
echo "RSRLIB is set to $RSRLIB"

```
EXERCISE 1

    1. Try changing the environment variable VERBOSE to 1 (True) or 0 (False) to see the effect.
    
    2. You can change the name of the directory where the object and material files are through the environment variable BPMSROOT. See if you can find what BPMSROOT is set to, and also see if yoiu can modify it.
    
```

**Answers below:**

In [ ]:
%%bash
# Answers to Exercise 1

#----------------------
# this part same as above
# test the init file
# set INIT script
source $BPMS/bin/local_init.sh
source $INIT
#----------------------

# 1.1: Try changing the environment variable VERBOSE to 1
# (True) and 0 (False) to see the effect.

# ANSWER
# this sets verbose mode and prints a message
# 'RATstart found ok' if it finds the librat RATstart
# executable
echo  "----set VERBOSE 1---"
export VERBOSE=1
source examples_init.sh
# it would be correct, in this instance to use
# export VERBOSE=1
# $INIT
# as there is no real need to source the file
# Above, the export in 'export VERBOSE=1' is not
# strictly needed if we source $INIT, but it is otherwise

# this turns off the verbose mode
# so no message is printed
echo "----set VERBOSE 0---"
export VERBOSE=0
source examples_init.sh
# it would be correct, in this instance to use
# export VERBOSE=0
# $INIT
# as there is no real need to source the file
# Above, the export in 'export VERBOSE=1' is not
# strictly needed if we source $INIT, but it is otherwise

# 1.2: You can change the name of the directory where the 
# object and material files are through the 
# environment variable BPMSROOT. 
# See if you can find what BPMSROOT is set to, 
# and also see if yoiu can modify it.

# ANSWER
# we need to see the value of the 
# environment variable BPMSROOT, but we need it 
# in this shell. So we *source* the file
# rather than running it.
echo "----get BPMSROOT---"
source examples_init.sh
echo "BPMSROOT is $BPMSROOT"
echo "----set BPMSROOT---"
# To change it, just set it before sourcing
export BPMSROOT="/tmp"
source examples_init.sh
echo "BPMSROOT is $BPMSROOT"

## Making some basic files

### `bash` constructs 

We will create the files we need as we go along, using `bash` shell `cat <<EOF > filename` syntax for multi-line data input. You may have noticed that we did this above in creating the `init` file. 

If we type:

    cat <<EOF > filename
    this is line 1
    this is line 2
    EOF
    
then a file called `filename` will be generated, containing the information up to the `EOF` marker:

    this is line 1
    this is line 2
   
   
In a similar fashion, if we type:

    cat <<<"this is line 1" > filename

(note the quotes!) then we can create a single line file with the contents:

    this is line 1
    
This is equivalent to:

    echo "this is line 1" | cat > filename
    

In [ ]:
%%bash

cat <<<"hello world" 

In [ ]:
%%bash

cat << EOF
hello world
EOF

    EXERCISE 2

        1. Use these shell constructs as appropriate to create a file called green.dat, with 2 columns of data, the first being wavelength in nm, the second being reflectance, representing a typical green spectrum, for samples at 450 nm, 550 nm and 650 nm.

        2. Use these shell constructs as appropriate to create a file called white.dat, with 2 columns of data, the first being wavelength in nm, the second being reflectance, representing a white spectrum, for samples at 450 nm, 550 nm and 650 nm.
        
**Answers below:**

In [ ]:
%%bash

# Answers to Exercise 2

# 2.1 green: high at 550 nm, low at 450 and 650 nm
# Its a multi-line file that we need, so
cat << EOF > $BPMS/obj/green_.dat
450 0.1
550 0.9
650 0.1
EOF

# 2.2 white: high at all
# Its a multi-line file that we need, so
cat << EOF > $BPMS/obj/white_.dat
450 0.1
550 0.9
650 0.1
EOF

# show results:
echo "========"
echo "green"
echo "========"
cat $BPMS/obj/green_.dat

echo "========"
echo "white"
echo "========"
cat $BPMS/obj/white_.dat

# be polite and tidy up
rm -f $BPMS/obj/green_.dat $BPMS/obj/white_.dat

### `first.obj` and associated files

Now we will use these constructs to put some text some files in that directory. We will be creating files that we need to run a radiative transfer simulation. We will look into what these mean and their formats later in the notes.

In [ ]:
%%bash

source examples_init.sh
# simple object file
# with green plane
# and sphere of radius 100 mm
# centred at (0,0,0)
cat <<EOF > $BPMSROOT/first.obj
# My first object file
mtllib plants.matlib.new
usemtl green 
v 0 0 0 
v 0 0 1 
plane -1 -2 
!{
usemtl white 
!{ 
v 0 0 0 
sph -1 1
!} 
!}
EOF

# wavelengths (nm)
cat <<EOF > $BPMSROOT/wavebands.dat
1 650
2 550
3 450
EOF

# spectrum for white
cat <<EOF > $BPMSROOT/white.dat
450 1
550 1
650 1
EOF

# spectrum for green
cat <<EOF > $BPMSROOT/green.dat
450 0.1
550 0.5
650 0.1
EOF

# library listing materials
cat <<EOF > $BPMSROOT/plants.matlib.new 
srm green green.dat
srm white white.dat
EOF

The scene object described in `first.obj` is a sphere of radius 1, centred at location `(0, 0, 0)`, as well as a plane with normal `(0, 0, 1)` passing through location `(0, 0, 0)`. The sphere is `white` and the plane is `green`.

### height map visualisation

In the first example, we produce a visualisation of the object using option `16`.

    16 cx cy cz sx sy nrows ncols rpp name : produce a height map in name
    
Option 16 produces an image dataset (in a rather old 'hips' format) that is a height map of the scene, produced by ray casting. Imagine a rectangle on an x-y plane of physical dimensions (sx, sy), centred at (cx, cy, cz). 

In [ ]:
%%bash
source examples_init.sh
# option 8
RATstart first.obj << EOF
16 0 0 4 4 4 200 200 1 $BPMS/obj/out.tmp.hips
EOF

We can use the python library `libhipl` from `RATlibUtils` to read (and write) hips format images:

In [ ]:
from RATlibUtils.libhipl import Hipl
import pylab as plt

f = BPMS+'/obj/out.tmp.hips'
plt.imshow(Hipl().read(f),cmap='gray')
plt.colorbar()

In the visualisation we can see the shere and plane as a height map. The height varies from 0 (the plane) to 1 (at the top of the sphere). The extent of the scene is `4 x 4` units, so we can confirm that the radius of the sphere is 1 unit by inspection.

### Ray tracing

And now another example.

We run this with the verbose flag set on running `RATstart`.

The input:

        1 3 0 1 1 1 1 1 1 0 1
        
(option `1`) sets the number of suns (3) followed by the sun direction vectors `(0, 1, 1)` `(1, 1, 1)` and `(1, 0, 1)` here. In practice, we tend to run simulations with a single illumination (sun) vector, but we illustrate using 3 here to encourage the use of this option.

The input:

        6 0 0 10 0 0 -1
        
(option `6`) sets up a ray tracing case, for a ray *from* location `(0 0 10)`, in direction `(0 0 -1)` (straight down).

We set the `-RATtolerance` command line option to a small number, relative to scene size objects: `0.0000001` is sufficient in this case.

We should be able to visualise this situation, and see that the ray will hit the sphere at `(0 0 1)`, so be of length 9 units.

In [ ]:
%%bash
source examples_init.sh

RATstart -RATtolerance 0.0000001 -RATsensor_wavebands wavebands.dat first.obj <<EOF
1 3 0 1 1 1 1 1 1 0 1
6 0 0 10 0 0 -1               
EOF


In the example above, we hit the sphere at its apex. The normal vector of the sphere at that point is `(0,0,1)` and the material reflectance (`white`) is 1.0 at all wavelengths. So we might notice that the `direct` reflectance above is always the cosine of the angle between the normal vector `(0,0,1)` and the solar vector.

We explore this in the code below by calculating the [vector dot product](https://mathworld.wolfram.com/DotProduct.html) of the sun and normal unit vectors. Note that the unit vectors have unit (1) length, so if we define them as vectors of arbitrary length (e.g. `(1, 1, 1)`) we need to normalise them (divide by the length, to give the unit length):

In [ ]:
import numpy as np
from numpy import linalg as LA

#### dot product for angle ####

sun = np.array([1.,1,1.])
# normalise vector
length = LA.norm(sun)
sunHat = sun/length

normal = np.array([0,0,1.])
# normalise vector
length = LA.norm(normal)
normalHat = normal/length

cosine = np.dot(sunHat,normalHat)
print("cosine of",sun,"=",cosine)

    EXERCISE 3
    
    Adapt the RATstart example above to have just a single sun vector. 
    
    Change the sun vector in the RATstart example, and use the python code above to predict what the direct reflectance will be. Do this for several examples. 
    
**Answers below:**

In [ ]:
%%bash
#
# set a single sun vector, here (0,0,1)
# so dot product = 1.0 and direct refl = 1
#
source examples_init.sh

RATstart -RATtolerance 0.0000001 -RATsensor_wavebands wavebands.dat first.obj <<EOF
1 1 0 0 1
6 0 0 10 0 0 -1               
EOF

In [ ]:
import numpy as np
from numpy import linalg as LA

#### dot product for angle ####

sun = np.array([0,0,1.])
# normalise vector
length = LA.norm(sun)
sunHat = sun/length

normal = np.array([0,0,1.])
# normalise vector
length = LA.norm(normal)
normalHat = normal/length

cosine = np.dot(sunHat,normalHat)
print("cosine of",sun,"=",cosine)

In [ ]:
%%bash
#
# set a single sun vector, here (0,1,1)
# so dot product = 0.707 (sqrt(2)) 
# and so direct refl = 0.707
#
source examples_init.sh

RATstart -RATtolerance 0.0000001 -RATsensor_wavebands wavebands.dat first.obj <<EOF
1 1 0 1 1
6 0 0 10 0 0 -1               
EOF

In [ ]:
import numpy as np
from numpy import linalg as LA

#### dot product for angle ####

sun = np.array([0,1,1.])
# normalise vector
length = LA.norm(sun)
sunHat = sun/length

normal = np.array([0,0,1.])
# normalise vector
length = LA.norm(normal)
normalHat = normal/length

cosine = np.dot(sunHat,normalHat)
print("cosine of",sun,"=",cosine)

### Summary

In this section, we have introduced the `RATstart` code, some information about the requirements for running the code. We generated some initialisation scripts to simplify setting up environment variables, generated a simple world object (sphere on a plane) and visualised this.